# 13 - Fashion Deep Learning Recommendation Systems



https://www.kaggle.com/datasets/paramaggarwal/fashion-product-images-small

In [93]:
import numpy as np
import pickle as pkl
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalMaxPool2D

from sklearn.neighbors import NearestNeighbors
import os
from numpy.linalg import norm

In [94]:
filenames = []
for file in os.listdir('/kaggle/input/fashion-product-images-small/images'):
    filenames.append(os.path.join('/kaggle/input/fashion-product-images-small/images', file))

In [95]:
len(filenames)

44441

In [96]:
model = ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))
model.trainable = False

model = tf.keras.models.Sequential([model,
                                   GlobalMaxPool2D()
                                   ])
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ ?                      │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling2d_4          │ ?                      │   0 (unbuilt) │
│ (GlobalMaxPooling2D)            │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,587,712 (89.98 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 23,587,712 (89.98 MB)

In [97]:
def extract_features_from_images(image_path, model):
    img = image.load_img(image_path, target_size=(224,224))
    img_array = image.img_to_array(img)
    img_expand_dim = np.expand_dims(img_array, axis=0)
    img_preprocess = preprocess_input(img_expand_dim)
    result = model.predict(img_preprocess).flatten()
    norm_result = result/norm(result)
    return norm_result

In [98]:
extract_features_from_images(filenames[0], model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


array([0.00043661, 0.        , 0.02227335, ..., 0.00194948, 0.01470532,
       0.01042836], dtype=float32)

In [99]:
image_features = []
for file in filenames:  # Tüm dosyaları işle
    features = extract_features_from_images(file, model)
    if features is not None:
        image_features.append(features)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step


[array([0.00043661, 0.        , 0.02227335, ..., 0.00194948, 0.01470532,
        0.01042836], dtype=float32),
 array([0.        , 0.05156705, 0.00386303, ..., 0.03591822, 0.02286726,
        0.06476431], dtype=float32),
 array([0.00700927, 0.05616754, 0.01070841, ..., 0.00207852, 0.03612409,
        0.00679433], dtype=float32),
 array([0.00359723, 0.00645735, 0.02641729, ..., 0.01571777, 0.03221679,
        0.01262881], dtype=float32),
 array([0.        , 0.01346459, 0.01379768, ..., 0.01387399, 0.02089634,
        0.02879267], dtype=float32)]

In [100]:
Image_features = pkl.dump(image_features, open('Images_features.pkl','wb'))

In [101]:
filenames = pkl.dump(filenames, open('filenames.pkl','wb'))

In [102]:
Image_features = pkl.load(open('Images_features.pkl','rb'))

In [103]:
filenames = pkl.load(open('filenames.pkl','rb'))

In [104]:
np.array(Image_features).shape

(5, 2048)

In [105]:
neighbors = NearestNeighbors(n_neighbors=5, algorithm='brute', metric='euclidean')

In [106]:
neighbors.fit(Image_features)

NearestNeighbors(algorithm='brute', metric='euclidean', n_neighbors=6)

In [107]:
input_image = extract_features_from_images('/kaggle/input/fashion-product-images-small/images/10008.jpg', model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step


In [108]:
distance,indices = neighbors.kneighbors([input_image])

ValueError: Expected n_neighbors <= n_samples,  but n_samples = 5, n_neighbors = 6

In [ ]:
indices[0]

In [ ]:
from IPython.display import Image
Image('/kaggle/input/fashion-product-images-small/images/10008.jpg')

In [ ]:
Image(filenames[indices[0][1]])

In [ ]:
Image(filenames[indices[0][2]])

In [ ]:
Image(filenames[indices[0][3]])

In [ ]:
Image(filenames[indices[0][4]])

In [ ]:
Create a streamlit app using this trained model 
and when I pick one item, it should recommend similar items using 
NearestNeighbors(n_neighbors=6, algorithm='brute', metric='euclidean')